In [1]:
import pandas as pd
import re
import pubchempy as pcp
import pandas as pd
import rdkit.Chem
import rdkit.Chem.AllChem
import rdkit.DataStructs

#### please download the DrugComb whole raw dataset from https://drugcomb.org/download/

In [2]:
df_dc = pd.read_csv("/home/cx79/scratch60/NCI/DB/summary_v_1_5.csv",encoding = "ISO-8859-1",dtype = str)
## remove na
df_dc_nadrop = df_dc.dropna(how='any')
subdf_dc = df_dc_nadrop

In [4]:
len(set(subdf_dc['cell_line_name']))

288

#### 1. Transform cell line name to cellosaurus ID (CVCL_) and DepMap ID

In [5]:
#We provide a function in access to expasy database, to get the CVCL ID by querying the cell name

In [6]:
raw_cell_name = list(set(subdf_dc['cell_line_name']))
for rawCellname in raw_cell_name:
    try:
        response = requests.get("https://web.expasy.org/cgi-bin/cellosaurus/search?input="+rawCellname)
        getID = re.findall(r'CVCL.*">', response.text)[0]
        getID = getID.split('"')[0]
        print(rawCellname, getID)
    except:
        pass

In [7]:
# Here we have provided a file that maps drugcomb cell line name to CVCL ID. This is the same as using the above function
drugcomb_cellosaurus = pd.read_csv("drugcomb_cellosaurus.csv",sep=',')

In [8]:
drugcomb_cellosaurus

,oldId,realID
0,VM-CUB-1,CVCL_1786
1,Huh-7,CVCL_0336
2,SNB-75,CVCL_1706
3,HT144,CVCL_0318
4,WM115,CVCL_0040
...,...,...
228,NCI-H3122,CVCL_5160
229,HCC1569,CVCL_1255
230,UO-31,CVCL_1911
231,MeWo,CVCL_0445


In [9]:
#We carefully examine the outputs from cellosaurus queried outputs. We manually corrected the wrong ones; added the missing one
drugcomb_cellosaurus['realID'][125] = 'CVCL_YQ49'
drugcomb_cellosaurus['realID'][60] = 'nan'
drugcomb_cellosaurus['realID'][223] = 'CVCL_0292'
drugcomb_cellosaurus['realID'][75] = 'CVCL_0105'
drugcomb_cellosaurus['realID'][162] = 'CVCL_0021'
drugcomb_cellosaurus['realID'][70] = 'CVCL_0132'
drugcomb_cellosaurus['realID'][19] = 'CVCL_1452'
drugcomb_cellosaurus['realID'][190] = 'CVCL_0031'
drugcomb_cellosaurus['realID'][52] = 'CVCL_0291'
drugcomb_cellosaurus['realID'][73] = 'CVCL_0465'
drugcomb_cellosaurus['realID'][106] = 'CVCL_0248'
drugcomb_cellosaurus = drugcomb_cellosaurus.append({'oldId': 'HT29','realID':'CVCL_0320'}, ignore_index=True)
drugcomb_cellosaurus = drugcomb_cellosaurus.dropna(how='any')

In [10]:
drugcomb_cellosaurus

,oldId,realID
0,VM-CUB-1,CVCL_1786
1,Huh-7,CVCL_0336
2,SNB-75,CVCL_1706
3,HT144,CVCL_0318
4,WM115,CVCL_0040
...,...,...
229,HCC1569,CVCL_1255
230,UO-31,CVCL_1911
231,MeWo,CVCL_0445
232,U3068MG,CVCL_IR89


In [11]:
# Next, we transform CCLV_ID(RRID) to DepMapID. The file "sample_info.csv"
# This was downloaded from the DepMap portal: https://depmap.org/portal/download/

In [12]:
sample_info = pd.read_csv('sample_info.csv')
sample_info = sample_info[['DepMap_ID','RRID']]
sample_info = sample_info.dropna(how='any')

In [13]:
sample_info

,DepMap_ID,RRID
0,ACH-000001,CVCL_0465
1,ACH-000002,CVCL_0002
2,ACH-000003,CVCL_0025
3,ACH-000004,CVCL_0001
4,ACH-000005,CVCL_2481
...,...,...
1801,ACH-002393,CVCL_1810
1802,ACH-002394,CVCL_0271
1803,ACH-002395,CVCL_1296
1804,ACH-002396,CVCL_5G89


In [15]:
realsubdf_dc = drugcomb_cellosaurus.merge(subdf_dc,left_on="oldId",right_on="cell_line_name")
# Dataframe which cells have a CCLV_ID
#realsubdf_dc

In [16]:
drugcomb_drug = set(realsubdf_dc['realID'])
CCLV_drug = set(sample_info['RRID'])
# These are cell lines that don't have a DepMapID
drugcomb_drug - (drugcomb_drug.intersection(CCLV_drug))

{'CVCL_0125',
 'CVCL_0219',
 'CVCL_0289',
 'CVCL_0395',
 'CVCL_0417',
 'CVCL_0B75',
 'CVCL_1384',
 'CVCL_1452',
 'CVCL_1570',
 'CVCL_1711',
 'CVCL_1813',
 'CVCL_2005',
 'CVCL_2088',
 'CVCL_2220',
 'CVCL_2H54',
 'CVCL_3799',
 'CVCL_4633',
 'CVCL_4791',
 'CVCL_5246',
 'CVCL_5310',
 'CVCL_5997',
 'CVCL_9827',
 'CVCL_A426',
 'CVCL_A442',
 'CVCL_A794',
 'CVCL_A8EZ',
 'CVCL_A8IW',
 'CVCL_AP82',
 'CVCL_AX39',
 'CVCL_IP22',
 'CVCL_IR58',
 'CVCL_IR61',
 'CVCL_IR67',
 'CVCL_IR73',
 'CVCL_IR89',
 'CVCL_IT41',
 'CVCL_JF18',
 'CVCL_JF68',
 'CVCL_M814',
 'CVCL_T887',
 'CVCL_VT28',
 'CVCL_VT33',
 'CVCL_Y053',
 'CVCL_YQ49',
 'CVCL_YX83',
 'CVCL_YX86',
 'CVCL_ZA06',
 'nan'}

In [17]:
# So finally, we obtain 184 cell lines which have both CCLV_ID and DepMapID
len(drugcomb_drug.intersection(CCLV_drug))

184

In [18]:
realsubdf_dc = realsubdf_dc.merge(sample_info, left_on='realID',right_on='RRID')

In [19]:
# Now we have mapped 184 cell line name to both IDs. The resulting dataframe has 525362 rows
realsubdf_dc

,oldId,realID,block_id,drug_row,drug_col,cell_line_name,study_name,tissue_name,conc_row_unit,conc_col_unit,...,synergy_zip,synergy_loewe,synergy_hsa,synergy_bliss,drug_row_clinical_phase,drug_col_clinical_phase,drug_row_target_name,drug_col_target_name,DepMap_ID,RRID
0,VM-CUB-1,CVCL_1786,1187287,AZD5363,KU-60019,VM-CUB-1,ASTRAZENECA,urinary_tract,uM,uM,...,-0.329321809194946,5.92559112725863,6.32323598003741,5.24513954350488,2,0,Rho-associated protein kinase 1; Rho-associate...,Serine-protein kinase ATM,ACH-000545,CVCL_1786
1,VM-CUB-1,CVCL_1786,1187301,AZD5363,AZD6738,VM-CUB-1,ASTRAZENECA,urinary_tract,uM,uM,...,-13.4548036550335,-22.5010258441113,-22.8113258572499,-21.7616990206227,2,2,Rho-associated protein kinase 1; Rho-associate...,Serine-protein kinase ATR; Serine/threonine-pr...,ACH-000545,CVCL_1786
2,VM-CUB-1,CVCL_1786,1187528,AZD5363,AZD-4877,VM-CUB-1,ASTRAZENECA,urinary_tract,uM,uM,...,39.6025220450632,-77.8979551538373,-40.2879401730772,29.9623708194271,2,2,Rho-associated protein kinase 1; Rho-associate...,Kinesin-like protein 1,ACH-000545,CVCL_1786
3,VM-CUB-1,CVCL_1786,1187552,AZD5363,Sapitinib,VM-CUB-1,ASTRAZENECA,urinary_tract,uM,uM,...,-7.84032977265941,0.150895555224506,1.47603779864746,-8.33016270978925,2,2,Rho-associated protein kinase 1; Rho-associate...,Receptor tyrosine-protein kinase erbB-3; Recep...,ACH-000545,CVCL_1786
4,VM-CUB-1,CVCL_1786,1187624,AZD5363,AZD4547,VM-CUB-1,ASTRAZENECA,urinary_tract,uM,uM,...,104.967699460057,-16.4017946571779,-0.776249738659982,146.089535158276,2,3,Rho-associated protein kinase 1; Rho-associate...,Fibroblast growth factor receptor 3; Fibroblas...,ACH-000545,CVCL_1786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525357,MeWo,CVCL_0445,1414780,ZM 336372,CHEMBL1254007,MeWo,FRIEDMAN,skin,uM,uM,...,-15.51657775437,-8.18366504773284,-47.4546017047357,-53.2051217717236,0,0,MAP kinase p38 alpha; Serine/threonine-protein...,Vascular endothelial growth factor receptor 2;...,ACH-000987,CVCL_0445
525358,MeWo,CVCL_0445,1414816,ZM 336372,Tipifarnib,MeWo,FRIEDMAN,skin,uM,uM,...,-7.18601260139362,-6.4046979318571,-18.9127303185778,-13.3089508855435,0,3,MAP kinase p38 alpha; Serine/threonine-protein...,Protein farnesyltransferase; Protein farnesylt...,ACH-000987,CVCL_0445
525359,MeWo,CVCL_0445,1414852,ZM 336372,112648-68-7,MeWo,FRIEDMAN,skin,uM,uM,...,-12.4293928301602,-10.6934180961554,-14.2853522779673,-13.9084149740677,0,0,MAP kinase p38 alpha; Serine/threonine-protein...,Aldehyde dehydrogenase 1A1; Telomerase reverse...,ACH-000987,CVCL_0445
525360,MeWo,CVCL_0445,1414888,ZM 336372,vasastrol,MeWo,FRIEDMAN,skin,uM,uM,...,-8.37772366579519,-2.45664955308483,-22.5798008488644,-29.4067097567556,0,\N,MAP kinase p38 alpha; Serine/threonine-protein...,\N,ACH-000987,CVCL_0445


#### 2. Transform drug names to uniform DrugBank IDs

In [21]:
###1. Allocate all unique drugs appearing in DrugComb

def get_uniquedrug(subsetdf):
    drug_list = []
    drug1_dc = subsetdf["drug_row"].unique()
    drug2_dc = subsetdf["drug_col"].unique()
    
    drug_list.append(set(drug1_dc))
    drug_list.append(set(drug2_dc))
    return drug_list

In [22]:
drug_list = get_uniquedrug(realsubdf_dc)
# One-Liner to union a list of sets
drug_list_set = set().union(*drug_list)

In [23]:
#In total there are 3127 drugs in the aforementioned cell line-cleaned dataset
len(drug_list_set)

3127

In [17]:
# 1. get pubchemID for each unique drug appearing in DrugComb by using PubChem API

def get_compounds(a_drug):
    return pcp.get_compounds(a_drug, 'name',listkey_count=1)

def get_pubchemID(drug_list_set):
    chemID_list = [ get_compounds(x)[0] if len(get_compounds(x))>0 else "" for x in drug_list_set ]
    print(x for x in drug_list_set)
    return chemID_list

chemID_list = get_pubchemID(drug_list_set)
pubchemIDmapping = pd.DataFrame({
    'drug_name': list(drug_list_set),
    'pubchemID': chemID_list
})
pubchemIDmapping['pubchemID'] = pubchemIDmapping['pubchemID'].astype(str)
# pubchemIDmapping.to_csv('chemID_drugcomb.csv')

In [25]:
# Here we have provided a mapping list from drug raw name to its pubchemID
pubchemIDmapping = pd.read_csv('chemID_drugcomb.csv')
pubchemIDmapping = pubchemIDmapping[['drug_name','pubchemID']]

In [26]:
# We manually corrected uncorrected ones
pubchemIDmapping['pubchemID'][12] = 'Compound(57519511)'
pubchemIDmapping['pubchemID'][446] = 'Compound(53396327)'
pubchemIDmapping['pubchemID'][554] = 'Compound(60865)'
pubchemIDmapping['pubchemID'][560] = 'Compound(15604015)'
pubchemIDmapping['pubchemID'][715] = 'Compound(5284419)'
pubchemIDmapping['pubchemID'][764] = 'Compound(363740391)'
pubchemIDmapping['pubchemID'][965] = 'Compound(163176602)'
pubchemIDmapping['pubchemID'][1565] = 'Compound(73265275)'
pubchemIDmapping['pubchemID'][1690] = 'Compound(73265237)'
pubchemIDmapping['pubchemID'][1861] = 'Compound(26041)'
pubchemIDmapping['pubchemID'][1883] = 'Compound(9578866)'
pubchemIDmapping['pubchemID'][2397] = 'Compound(6445562)'
pubchemIDmapping['pubchemID'][3038] = 'Compound(6918664)'

In [28]:
pubchemIDmapping = pubchemIDmapping[pubchemIDmapping['pubchemID'].notna()]
pubchemIDmapping

,drug_name,pubchemID
0,primaquine diphosphate,Compound(6135)
1,Gbr 12935,Compound(3456)
2,GW7647,Compound(3392731)
3,153436-54-5,Compound(4705)
4,Rizatriptan benzoate,Compound(77997)
...,...,...
3122,KETOTIFEN FUMARATE,Compound(5282408)
3123,Ki8751,Compound(11317348)
3124,Fesoterodine fumarate,Compound(9849808)
3125,Rifaximin,Compound(6436173)


In [29]:
#re.search(r'\((.*?)\)','Compound(148124)').group(1)''
# split Coumponud ID to pure integer number
def split_it(compound):
    return int(re.search(r'\((.*?)\)', compound).group(1))

pubchemIDmapping['cleanID'] = pubchemIDmapping['pubchemID'].apply(lambda x: split_it(x))
pubchemIDmapping['cleanID'] = pubchemIDmapping['cleanID'].astype(int)

In [30]:
# Since we want to get DrugBankID to each drug, we dowloaded SDF structures for the whole DrugBank database. 
# On the other hand, we also get SDF structure for each PubChem Compound.
# Calculate PubChem compound similarity against all drugbank drugs, we're done!

In [31]:
def get_pubchemsdf(cleanID):
    pcp.download('SDF', str(cleanID)+'.sdf', cleanID, 'cid', overwrite=True)
    
# for _ in list(pubchemIDmapping['cleanID']):
#     try:
#         get_pubchemsdf(_)
#     except:
#         print(_)
#         pass

In [34]:
# Read SDF File for pubchem compounds
supplier = rdkit.Chem.SDMolSupplier('structures.sdf')
molecules = [mol for mol in supplier if mol is not None]
# Read SDF File for Drugbank compounds
drugcomb_supplier = rdkit.Chem.SDMolSupplier('drugcomb_structures.sdf')
drugcomb_molecules = [mol for mol in drugcomb_supplier if mol is not None]

RDKit ERROR: [15:13:42] ERROR: Explicit valence for atom # 7 K, 2, is greater than permitted
[15:13:42] ERROR: Explicit valence for atom # 7 K, 2, is greater than permitted
[15:14:14] Explicit valence for atom # 13 Cl, 5, is greater than permitted
RDKit ERROR: [15:14:14] Explicit valence for atom # 13 Cl, 5, is greater than permitted
RDKit ERROR: [15:14:14] ERROR: Could not sanitize molecule ending on line 289199
[15:14:14] ERROR: Could not sanitize molecule ending on line 289199
RDKit ERROR: [15:14:14] ERROR: Explicit valence for atom # 13 Cl, 5, is greater than permitted
[15:14:14] ERROR: Explicit valence for atom # 13 Cl, 5, is greater than permitted
[15:14:14] Explicit valence for atom # 19 O, 3, is greater than permitted
RDKit ERROR: [15:14:14] Explicit valence for atom # 19 O, 3, is greater than permitted
RDKit ERROR: [15:14:14] ERROR: Could not sanitize molecule ending on line 399275
[15:14:14] ERROR: Could not sanitize molecule ending on line 399275
RDKit ERROR: [15:14:14] ERRO

In [35]:
len(drugcomb_molecules)

3043

In [36]:
# Calculate fingerprints for drugbank
fingerprints = dict()
for mol in molecules:
    drugbank_id = mol.GetProp('DATABASE_ID')
    fingerprint = rdkit.Chem.AllChem.GetMorganFingerprint(mol, 2)
    fingerprints[drugbank_id] = fingerprint

In [37]:
drugcomb_fingerprints = dict()
for mol in drugcomb_molecules:
    pubchem_id = mol.GetProp('PUBCHEM_COMPOUND_CID')
    fingerprint = rdkit.Chem.AllChem.GetMorganFingerprint(mol, 2)
    drugcomb_fingerprints[pubchem_id] = fingerprint

In [50]:
# Calculate pairwise compound similarities
similarity_rows = list()
for id0, fp0 in fingerprints.items():
    for id1,fp1 in drugcomb_fingerprints.items():
        similarity = rdkit.DataStructs.DiceSimilarity(fp0, fp1)
        similarity = round(similarity, 4)
        similarity_rows.append([id0, id1, similarity])

In [51]:
similarity_df = pd.DataFrame(similarity_rows, columns=['compound0', 'compound1', 'similarity'])

In [39]:
#similarity_df.to_csv("similarity_drugcomb.tsv", sep='\t')
similarity_df = pd.read_csv("similarity_drugcomb.tsv", sep='\t')

In [59]:
len(similarity_df)

34355470

In [42]:
pd.set_option('display.max_rows', 200)
# For each drug, the highest simlarity is selected
idx = similarity_df.groupby(by=["compound1"])['similarity'].transform(max) == similarity_df['similarity']
similarity_df = similarity_df[idx]
similarity_df = similarity_df.drop_duplicates(subset=['compound1'])

In [43]:
# Here we pre-define when the highest similarity >= 0.8, we could say a PubChemID is mapping to a DrugBank drug
confident_similarity_df = similarity_df[similarity_df['similarity']>=0.8]
confident_similarity_df

,Unnamed: 0,compound0,compound1,similarity
21871,21871,DB00080,16134395,1.0000
26190,26190,DB00091,5284373,1.0000
31947,31947,DB00104,448601,1.0000
41453,41453,DB00115,5311498,0.9355
42307,42307,DB00115,73265345,0.9355
...,...,...,...,...
34055141,34055141,DB16305,25022340,1.0000
34127577,34127577,DB16346,123723,1.0000
34221434,34221434,DB16567,9576185,0.9925
34320126,34320126,DB16702,3326,1.0000


In [44]:
notconfident_similarity_df = similarity_df[similarity_df['similarity']<0.8]
notconfident_similarity_df.head(20)

,Unnamed: 0,compound0,compound1,similarity
230846,230846,DB00179,72300,0.5517
240514,240514,DB00183,108187,0.5252
243510,243510,DB00184,10434468,0.5932
297473,297473,DB00201,6241,0.6981
323127,323127,DB00210,16124726,0.6264
343506,343506,DB00216,73265274,0.6012
393135,393135,DB00233,16367,0.6744
400192,400192,DB00235,4593,0.7475
458470,458470,DB00254,54686183,0.5670
473577,473577,DB00259,5324,0.7576


In [45]:
pubchemIDmapping['cleanID'] = pubchemIDmapping['cleanID'].astype(int)
confident_similarity_df['compound1'] = confident_similarity_df['compound1'].astype(int)

/gpfs/ycga/project/christakis/cx79/conda_envs/meta/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [46]:
confident_similarity =  pd.merge(pubchemIDmapping, confident_similarity_df, left_on=['cleanID'],right_on=['compound1'])

In [47]:
confident_similarity[confident_similarity.duplicated(subset=['drug_name'])]
#2163 has a matched Morgan fingerprint to a DrugBank compound 
confident_similarity

,drug_name,pubchemID,cleanID,Unnamed: 0,compound0,compound1,similarity
0,primaquine diphosphate,Compound(6135),6135,2923567,DB01087,6135,0.8438
1,153436-54-5,Compound(4705),4705,9243218,DB03365,4705,0.9048
2,Rizatriptan benzoate,Compound(77997),77997,2534600,DB00953,77997,0.8227
3,triamterene,Compound(5546),5546,844971,DB00384,5546,1.0000
4,DOXOFYLLINE,Compound(50942),50942,22723809,DB09273,50942,1.0000
...,...,...,...,...,...,...,...
2158,Tariquidar,Compound(148201),148201,14789461,DB06240,148201,1.0000
2159,KETOTIFEN FUMARATE,Compound(5282408),5282408,2436232,DB00920,5282408,0.8649
2160,Fesoterodine fumarate,Compound(9849808),9849808,15461397,DB06702,9849808,0.8913
2161,Rifaximin,Compound(6436173),6436173,3325304,DB01220,6436173,1.0000


In [48]:
temp1_dc = pd.merge(realsubdf_dc, confident_similarity, left_on=['drug_row'],right_on=['drug_name'], how='inner')
temp2_dc = pd.merge(temp1_dc, confident_similarity, left_on=['drug_col'],right_on=['drug_name'], how='inner')
#temp2_dc.to_csv('drugcomb_trueset.csv')

#### Now we have a sub-cleaned DrugComb dataset. For the 83% of the combinations (30 5986 rows), there is only a single record reporting Loewe score. 6 3319 rows (16371 trios) have more than 1 replicate record reporting Loewe scores. For the replicated trios, we summarize over the source of study. We filter out trios which report synergism/antagonism in different studies. 

In [49]:
temp2_dc

,oldId,realID,block_id,drug_row,drug_col,cell_line_name,study_name,tissue_name,conc_row_unit,conc_col_unit,...,compound0_x,compound1_x,similarity_x,drug_name_y,pubchemID_y,cleanID_y,Unnamed: 0_y,compound0_y,compound1_y,similarity_y
0,VM-CUB-1,CVCL_1786,1187301,AZD5363,AZD6738,VM-CUB-1,ASTRAZENECA,urinary_tract,uM,uM,...,DB12218,25227436,1.0,AZD6738,Compound(54761306),54761306,31585328,DB14917,54761306,0.8659
1,SW780,CVCL_1728,1187297,AZD5363,AZD6738,SW780,ASTRAZENECA,urinary_tract,uM,uM,...,DB12218,25227436,1.0,AZD6738,Compound(54761306),54761306,31585328,DB14917,54761306,0.8659
2,KU-19-19,CVCL_1344,1187294,AZD5363,AZD6738,KU-19-19,ASTRAZENECA,urinary_tract,uM,uM,...,DB12218,25227436,1.0,AZD6738,Compound(54761306),54761306,31585328,DB14917,54761306,0.8659
3,T24,CVCL_0554,1187298,AZD5363,AZD6738,T24,ASTRAZENECA,urinary_tract,uM,uM,...,DB12218,25227436,1.0,AZD6738,Compound(54761306),54761306,31585328,DB14917,54761306,0.8659
4,BFTC-905,CVCL_1083,1187290,AZD5363,AZD6738,BFTC-905,ASTRAZENECA,urinary_tract,uM,uM,...,DB12218,25227436,1.0,AZD6738,Compound(54761306),54761306,31585328,DB14917,54761306,0.8659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369350,COLO 205,CVCL_0218,1179718,5Z-7-Oxozeaenol,Doramapimod,COLO 205,FLOBAK,large_intestine,ÂµM,ÂµM,...,DB07788,9863776,1.0,Doramapimod,Compound(156422),156422,8317047,DB03044,156422,1.0000
369351,COLO 205,CVCL_0218,1179889,5Z-7-Oxozeaenol,Doramapimod,COLO 205,FLOBAK,large_intestine,ÂµM,ÂµM,...,DB07788,9863776,1.0,Doramapimod,Compound(156422),156422,8317047,DB03044,156422,1.0000
369352,COLO 205,CVCL_0218,1180060,5Z-7-Oxozeaenol,Doramapimod,COLO 205,FLOBAK,large_intestine,ÂµM,ÂµM,...,DB07788,9863776,1.0,Doramapimod,Compound(156422),156422,8317047,DB03044,156422,1.0000
369353,MDA-MB-468,CVCL_0419,1181577,5Z-7-Oxozeaenol,Doramapimod,MDA-MB-468,FLOBAK,breast,ÂµM,ÂµM,...,DB07788,9863776,1.0,Doramapimod,Compound(156422),156422,8317047,DB03044,156422,1.0000


In [50]:
len(set(temp2_dc['realID']))

183

In [51]:
temp_list = []
temp_list.append(temp2_dc['compound0_x'].to_list())
temp_list.append(temp2_dc['compound0_y'].to_list())
len(set().union(*temp_list))

1349

In [13]:
#TGSA_drugcomb = pd.read_csv('/home/cx79/scratch60/GraphSynergy_pivotals/data/DrugComb-trueset/drug_combinations_whole_TGSA.csv')

#### 3. remove duplicates

In [52]:
#dc_df =  pd.read_csv('drugcomb_trueset.csv')
dc_df = temp2_dc

In [53]:
len(dc_df)

369355

In [54]:
dc_df['synergistic'] = [0] * len(dc_df)
dc_df = dc_df[dc_df['synergy_loewe'] != '\\N']
dc_df['synergy_loewe'] = dc_df['synergy_loewe'].astype(float)
dc_df.loc[dc_df['synergy_loewe'] > 0, 'synergistic'] = 1

/gpfs/ycga/project/christakis/cx79/conda_envs/meta/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/gpfs/ycga/project/christakis/cx79/conda_envs/meta/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [55]:
def combine_triplets(test_set):
    combined_list =[]
    for index, row in test_set.iterrows():
        combined_list.append((row['realID'],row['compound0_x'],row['compound0_y']))

    test_set['combined'] = combined_list
    return test_set

In [56]:
combined_dc_df = combine_triplets(dc_df)

/gpfs/ycga/project/christakis/cx79/conda_envs/meta/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [57]:
#combined_dc_df[combined_dc_df.duplicated(subset='combined')].to_csv('duplciates_in_dc.csv')
nondup = combined_dc_df[~combined_dc_df.duplicated(subset='combined',keep=False)]
nondup

,oldId,realID,block_id,drug_row,drug_col,cell_line_name,study_name,tissue_name,conc_row_unit,conc_col_unit,...,similarity_x,drug_name_y,pubchemID_y,cleanID_y,Unnamed: 0_y,compound0_y,compound1_y,similarity_y,synergistic,combined
0,VM-CUB-1,CVCL_1786,1187301,AZD5363,AZD6738,VM-CUB-1,ASTRAZENECA,urinary_tract,uM,uM,...,1.0,AZD6738,Compound(54761306),54761306,31585328,DB14917,54761306,0.8659,0,"(CVCL_1786, DB12218, DB14917)"
1,SW780,CVCL_1728,1187297,AZD5363,AZD6738,SW780,ASTRAZENECA,urinary_tract,uM,uM,...,1.0,AZD6738,Compound(54761306),54761306,31585328,DB14917,54761306,0.8659,0,"(CVCL_1728, DB12218, DB14917)"
2,KU-19-19,CVCL_1344,1187294,AZD5363,AZD6738,KU-19-19,ASTRAZENECA,urinary_tract,uM,uM,...,1.0,AZD6738,Compound(54761306),54761306,31585328,DB14917,54761306,0.8659,0,"(CVCL_1344, DB12218, DB14917)"
3,T24,CVCL_0554,1187298,AZD5363,AZD6738,T24,ASTRAZENECA,urinary_tract,uM,uM,...,1.0,AZD6738,Compound(54761306),54761306,31585328,DB14917,54761306,0.8659,0,"(CVCL_0554, DB12218, DB14917)"
4,BFTC-905,CVCL_1083,1187290,AZD5363,AZD6738,BFTC-905,ASTRAZENECA,urinary_tract,uM,uM,...,1.0,AZD6738,Compound(54761306),54761306,31585328,DB14917,54761306,0.8659,0,"(CVCL_1083, DB12218, DB14917)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369294,SMS-CTR,CVCL_A770,462367,CHEMBL3348822,ceritinib,SMS-CTR,YOHE,soft_tissue,uM,uM,...,1.0,ceritinib,Compound(57379345),57379345,22182559,DB09063,57379345,1.0000,1,"(CVCL_A770, DB01590, DB09063)"
369295,RD,CVCL_1649,462553,Omipalisib,Omipalisib,RD,YOHE,soft_tissue,uM,uM,...,1.0,Omipalisib,Compound(25167777),25167777,26891947,DB12703,25167777,1.0000,0,"(CVCL_1649, DB12703, DB12703)"
369296,SMS-CTR,CVCL_A770,462373,Omipalisib,Omipalisib,SMS-CTR,YOHE,soft_tissue,uM,uM,...,1.0,Omipalisib,Compound(25167777),25167777,26891947,DB12703,25167777,1.0000,1,"(CVCL_A770, DB12703, DB12703)"
369297,RD,CVCL_1649,462556,CHEMBL3348822,Omipalisib,RD,YOHE,soft_tissue,uM,uM,...,1.0,Omipalisib,Compound(25167777),25167777,26891947,DB12703,25167777,1.0000,0,"(CVCL_1649, DB01590, DB12703)"


In [58]:
dupl = combined_dc_df[combined_dc_df.duplicated(subset='combined',keep=False)]
dupl

,oldId,realID,block_id,drug_row,drug_col,cell_line_name,study_name,tissue_name,conc_row_unit,conc_col_unit,...,similarity_x,drug_name_y,pubchemID_y,cleanID_y,Unnamed: 0_y,compound0_y,compound1_y,similarity_y,synergistic,combined
95,SW780,CVCL_1728,1195616,AZD4547,AZD6738,SW780,ASTRAZENECA,urinary_tract,uM,uM,...,1.0,AZD6738,Compound(54761306),54761306,31585328,DB14917,54761306,0.8659,1,"(CVCL_1728, DB12247, DB14917)"
102,RT-112,CVCL_1670,1195615,AZD4547,AZD6738,RT-112,ASTRAZENECA,urinary_tract,uM,uM,...,1.0,AZD6738,Compound(54761306),54761306,31585328,DB14917,54761306,0.8659,0,"(CVCL_1670, DB12247, DB14917)"
258,HCC1187,CVCL_1247,1192592,Gefitinib,AZD6738,HCC1187,ASTRAZENECA,breast,uM,uM,...,1.0,AZD6738,Compound(54761306),54761306,31585328,DB14917,54761306,0.8659,0,"(CVCL_1247, DB00317, DB14917)"
259,HCC1187,CVCL_1247,1192593,Gefitinib,AZD6738,HCC1187,ASTRAZENECA,breast,uM,uM,...,1.0,AZD6738,Compound(54761306),54761306,31585328,DB14917,54761306,0.8659,0,"(CVCL_1247, DB00317, DB14917)"
260,HCC1187,CVCL_1247,1192594,Gefitinib,AZD6738,HCC1187,ASTRAZENECA,breast,uM,uM,...,1.0,AZD6738,Compound(54761306),54761306,31585328,DB14917,54761306,0.8659,0,"(CVCL_1247, DB00317, DB14917)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369350,COLO 205,CVCL_0218,1179718,5Z-7-Oxozeaenol,Doramapimod,COLO 205,FLOBAK,large_intestine,ÂµM,ÂµM,...,1.0,Doramapimod,Compound(156422),156422,8317047,DB03044,156422,1.0000,0,"(CVCL_0218, DB07788, DB03044)"
369351,COLO 205,CVCL_0218,1179889,5Z-7-Oxozeaenol,Doramapimod,COLO 205,FLOBAK,large_intestine,ÂµM,ÂµM,...,1.0,Doramapimod,Compound(156422),156422,8317047,DB03044,156422,1.0000,0,"(CVCL_0218, DB07788, DB03044)"
369352,COLO 205,CVCL_0218,1180060,5Z-7-Oxozeaenol,Doramapimod,COLO 205,FLOBAK,large_intestine,ÂµM,ÂµM,...,1.0,Doramapimod,Compound(156422),156422,8317047,DB03044,156422,1.0000,0,"(CVCL_0218, DB07788, DB03044)"
369353,MDA-MB-468,CVCL_0419,1181577,5Z-7-Oxozeaenol,Doramapimod,MDA-MB-468,FLOBAK,breast,ÂµM,ÂµM,...,1.0,Doramapimod,Compound(156422),156422,8317047,DB03044,156422,1.0000,0,"(CVCL_0419, DB07788, DB03044)"


In [45]:
# groupdicts = dupl.groupby(by=["combined"]).groups
# groupdicts

In [59]:
def summarize_dupl(dupl):
    lists_ix = []
    groupdicts = dupl.groupby(by=["combined"]).groups
    for key, ind_list in groupdicts.items():
        subdf = dupl.loc[ind_list]
        if len(subdf['study_name']) <= 1:
            ix = subdf.drop_duplicates(subset=['combined']).index[0]
        
        elif subdf['synergistic'].mean() == 0 or subdf['synergistic'].mean() == 1:
            ## keep either row
            ix = subdf.drop_duplicates(subset=['combined']).index[0]
        else:
            continue
        lists_ix.append(ix)
    
    return dupl.loc[lists_ix]

In [60]:
dupl_slim = summarize_dupl(dupl)

In [63]:
dupl_slim

,oldId,realID,block_id,drug_row,drug_col,cell_line_name,study_name,tissue_name,conc_row_unit,conc_col_unit,...,similarity_x,drug_name_y,pubchemID_y,cleanID_y,Unnamed: 0_y,compound0_y,compound1_y,similarity_y,synergistic,combined
223214,K-562,CVCL_0004,289684,CHEMBL17639,ADM hydrochloride,K-562,ALMANAC,haematopoietic_and_lymphoid,uM,uM,...,1.0000,ADM hydrochloride,Compound(32874),32874,1023615,DB00445,32874,0.9953,0,"(CVCL_0004, DB00242, DB00445)"
224237,K-562,CVCL_0004,161172,5-Fluoro-2'-deoxyuridine,ADM hydrochloride,K-562,ALMANAC,haematopoietic_and_lymphoid,uM,uM,...,1.0000,ADM hydrochloride,Compound(32874),32874,1023615,DB00445,32874,0.9953,0,"(CVCL_0004, DB00322, DB00445)"
221606,K-562,CVCL_0004,199141,Zoledronic acid,ADM hydrochloride,K-562,ALMANAC,haematopoietic_and_lymphoid,uM,uM,...,1.0000,ADM hydrochloride,Compound(32874),32874,1023615,DB00445,32874,0.9953,0,"(CVCL_0004, DB00399, DB00445)"
225847,K-562,CVCL_0004,223458,122111-05-1,ADM hydrochloride,K-562,ALMANAC,haematopoietic_and_lymphoid,uM,uM,...,0.9897,ADM hydrochloride,Compound(32874),32874,1023615,DB00445,32874,0.9953,0,"(CVCL_0004, DB00441, DB00445)"
231410,K-562,CVCL_0004,434620,ADM hydrochloride,NSC256439,K-562,ALMANAC,haematopoietic_and_lymphoid,uM,uM,...,0.9953,NSC256439,Compound(3685),3685,3193352,DB01177,3685,1.0000,0,"(CVCL_0004, DB00445, DB01177)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369237,EW-8,CVCL_V618,461857,ZINC34894448,PF-3758309,EW-8,NCATS_ES(NAMPT+PARP),bone,uM,uM,...,1.0000,PF-3758309,Compound(25227462),25227462,24542764,DB11775,25227462,1.0000,1,"(CVCL_V618, DB12021, DB11775)"
369093,EW-8,CVCL_V618,461846,MK-5108,Doxorubicin Hydrochloride,EW-8,NCATS_ES(NAMPT+PARP),bone,uM,uM,...,1.0000,Doxorubicin Hydrochloride,Compound(443939),443939,1023908,DB00445,443939,0.9953,1,"(CVCL_V618, DB12556, DB00445)"
7210,EW-8,CVCL_V618,461874,10356-76-0,Olaparib,EW-8,NCATS_ES(NAMPT+PARP),bone,uM,uM,...,1.0000,Olaparib,Compound(23725625),23725625,22211694,DB09074,23725625,1.0000,0,"(CVCL_V618, DB12957, DB09074)"
256448,EW-8,CVCL_V618,461873,10356-76-0,Niraparib,EW-8,NCATS_ES(NAMPT+PARP),bone,uM,uM,...,1.0000,Niraparib,Compound(24958200),24958200,24588355,DB11793,24958200,1.0000,0,"(CVCL_V618, DB12957, DB11793)"


In [61]:
combined_dc_df_NoDup = pd.concat([dupl_slim, nondup])

In [62]:
combined_dc_df_NoDup

,oldId,realID,block_id,drug_row,drug_col,cell_line_name,study_name,tissue_name,conc_row_unit,conc_col_unit,...,similarity_x,drug_name_y,pubchemID_y,cleanID_y,Unnamed: 0_y,compound0_y,compound1_y,similarity_y,synergistic,combined
223214,K-562,CVCL_0004,289684,CHEMBL17639,ADM hydrochloride,K-562,ALMANAC,haematopoietic_and_lymphoid,uM,uM,...,1.0000,ADM hydrochloride,Compound(32874),32874,1023615,DB00445,32874,0.9953,0,"(CVCL_0004, DB00242, DB00445)"
224237,K-562,CVCL_0004,161172,5-Fluoro-2'-deoxyuridine,ADM hydrochloride,K-562,ALMANAC,haematopoietic_and_lymphoid,uM,uM,...,1.0000,ADM hydrochloride,Compound(32874),32874,1023615,DB00445,32874,0.9953,0,"(CVCL_0004, DB00322, DB00445)"
221606,K-562,CVCL_0004,199141,Zoledronic acid,ADM hydrochloride,K-562,ALMANAC,haematopoietic_and_lymphoid,uM,uM,...,1.0000,ADM hydrochloride,Compound(32874),32874,1023615,DB00445,32874,0.9953,0,"(CVCL_0004, DB00399, DB00445)"
225847,K-562,CVCL_0004,223458,122111-05-1,ADM hydrochloride,K-562,ALMANAC,haematopoietic_and_lymphoid,uM,uM,...,0.9897,ADM hydrochloride,Compound(32874),32874,1023615,DB00445,32874,0.9953,0,"(CVCL_0004, DB00441, DB00445)"
231410,K-562,CVCL_0004,434620,ADM hydrochloride,NSC256439,K-562,ALMANAC,haematopoietic_and_lymphoid,uM,uM,...,0.9953,NSC256439,Compound(3685),3685,3193352,DB01177,3685,1.0000,0,"(CVCL_0004, DB00445, DB01177)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369294,SMS-CTR,CVCL_A770,462367,CHEMBL3348822,ceritinib,SMS-CTR,YOHE,soft_tissue,uM,uM,...,1.0000,ceritinib,Compound(57379345),57379345,22182559,DB09063,57379345,1.0000,1,"(CVCL_A770, DB01590, DB09063)"
369295,RD,CVCL_1649,462553,Omipalisib,Omipalisib,RD,YOHE,soft_tissue,uM,uM,...,1.0000,Omipalisib,Compound(25167777),25167777,26891947,DB12703,25167777,1.0000,0,"(CVCL_1649, DB12703, DB12703)"
369296,SMS-CTR,CVCL_A770,462373,Omipalisib,Omipalisib,SMS-CTR,YOHE,soft_tissue,uM,uM,...,1.0000,Omipalisib,Compound(25167777),25167777,26891947,DB12703,25167777,1.0000,1,"(CVCL_A770, DB12703, DB12703)"
369297,RD,CVCL_1649,462556,CHEMBL3348822,Omipalisib,RD,YOHE,soft_tissue,uM,uM,...,1.0000,Omipalisib,Compound(25167777),25167777,26891947,DB12703,25167777,1.0000,0,"(CVCL_1649, DB01590, DB12703)"
